In [278]:
# for scrap_steam250
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output

# for data preprocessing & analysis
import pandas as pd
from datetime import datetime as dt

# for scrap_steamspy
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
top_rts_games_site = 'https://steam250.com/tag/rts'
steamspy_site = 'https://steamspy.com/tag/RTS'

In [398]:
request = requests.get(top_rts_games_site)
soup = BeautifulSoup(request.content, "html.parser")

In [405]:
def scrap_steam250(soup):
    res_dict = {}
    title = []
    release_date = []
    platforms = []
    score = []
    rating = []
    votes = []
    imgs = []
    
    for i in range(1, 151):
        game = soup.find(id=i)
        
        imgs.append(game.find('img').get('data-src'))
        title_curr = game.find('span', {'class' : 'title'}).find('a').text
        #needed, cause some titles have spec characters, that r not recognized by steamspy
        title.append(re.sub('\W+',' ', title_curr))
        try:
            release_date.append(game.find('span', {'class' : 'date'}).get('title'))
        except:
            release_date.append('no date')
        platforms.append([x.get('title') for x in game.find('span', {'class' : 'platforms'}).contents])
        score.append(game.find('span', {'class': 'score'}).text)
        rating.append(game.find('span', {'class': 'rating'}).text)
        votes.append(game.find('span', {'class': 'votes'}).text) 
        
        print('Processed %.2f%% of data' %(len(imgs)/150*100))
        clear_output(wait=True)

        
    res_dict['Game'] = title
    res_dict['Img'] = imgs
    res_dict['Release_Date'] = release_date
    res_dict['Platforms'] = platforms
    res_dict['Score'] = score
    res_dict['Rating'] = rating
    res_dict['Votes'] = votes
    
    return res_dict

In [286]:
res_dict = scrap_steam250(soup)

Processed 100.00% of data


In [260]:
# df = pd.DataFrame(res_dict)
# df

In [175]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get(steamspy_site)

C:\Users\Nastya\AppData\Local\Temp\ipykernel_8732\2361557983.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


In [284]:
games = res_dict['Game']
def scrap_steamspy(driver, games):
    game_links_steamspy = []
#     no_info_games = []
    owners = []
    
    search = driver.find_element(By.XPATH,"//input[@type='search']")
    
    for game in games:
        search.clear()
        search.send_keys(game)
        try:
            link_elem = driver.find_element(By.XPATH, "//a[starts-with (@href, '/app/')]")
            game_links_steamspy.append(link_elem.get_attribute('href'))
            owners_elem = driver.find_element(By.CLASS_NAME, 'sorting_1')
            owners.append(owners_elem.text)
        except:
#             no_info_games.append(game)
            game_links_steamspy.append('-')
            owners.append('-')
        search.clear()
        print('Processed %.2f%% of data' %(len(game_links_steamspy)/len(games)*100))
        clear_output(wait=True)
    
    driver.quit()
    return game_links_steamspy, owners

In [289]:
game_links_steamspy, owners = scrap_steamspy(driver, games)

Processed 100.00% of data


In [409]:
res_dict['Link_Steamspy'] = game_links_steamspy
res_dict['Owners'] = owners

In [411]:
df = pd.DataFrame(res_dict)

In [287]:
df[df['Link_Steamspy'] == '-']

,Game,Release_Date,Platforms,Score,Rating,Votes,Link_Steamspy,Owners
10,Door Kickers 2 Task Force North,3 Nov 2020,[Windows],8.36,97%,"6,422 votes",-,-
43,Heroes of the Three Kingdoms 2,12 Nov 2020,[Windows],8.07,95%,"1,276 votes",-,-
63,Element TD 2 Tower Defense,2 Apr 2021,[Windows],7.91,92%,"2,282 votes",-,-
64,From the Depths,6 Nov 2020,"[Windows, Mac, Linux]",7.90,90%,"9,763 votes",-,-
71,Banana Drama,25 Jul 2022,"[Windows, Mac, Linux]",7.86,97%,207 votes,-,-
74,ICBM,17 Nov 2020,[Windows],7.85,92%,"1,303 votes",-,-
75,Heroes of the Three Kingdoms,12 Nov 2020,[Windows],7.84,93%,"1,050 votes",-,-
97,112 Operator,23 Apr 2020,"[Windows, Mac]",7.74,89%,"4,545 votes",-,-
115,Crustacean Nations,19 Aug 2022,[Windows],7.69,94%,262 votes,-,-
116,Space Raiders in Space,8 Dec 2020,[Windows],7.68,96%,112 votes,-,-


In [277]:
df[df['Release_Date'] == 'no date']

,Game,Release_Date,Platforms,Score,Rating,Votes,Link_Steamspy,Owners
111,Stronghold 2 Steam Edition,no date,[Windows],7.69,88%,"4,129 votes",https://steamspy.com/app/40960,"500,000 .. 1,000,000"


In [288]:
# df.to_csv(r'...\games.csv')

In [413]:
df.loc[df.Game == 'Stronghold 2 Steam Edition','Release_Date'] = '5 Oct 2017'

In [414]:
df['Rating'] = df['Rating'].str.replace('%', '')
df['Votes'] = df['Votes'].str.replace('votes', '')

In [418]:
df['Rating'] = df['Rating'].astype(int)
df['Score'] = df['Score'].astype(float)
df['Release_Date'] = pd.to_datetime(df['Release_Date'], format='%d %b %Y')
df['Days_Since_Release'] = (dt.now() - df.Release_Date).dt.days
df.Owners = df.Owners.str.replace(',', '')
df = df[~(df.Owners == '-')]
df[['OwnersMin', 'OwnersMax']] = df.Owners.str.split(' .. ', expand=True).astype(int)

df.Votes = df.Votes.str.replace(',', '').astype(int)

C:\Users\Nastya\AppData\Local\Temp\ipykernel_8732\3574792755.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['OwnersMin', 'OwnersMax']] = df.Owners.str.split(' .. ', expand=True).astype(int)
C:\Users\Nastya\AppData\Local\Temp\ipykernel_8732\3574792755.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['OwnersMin', 'OwnersMax']] = df.Owners.str.split(' .. ', expand=True).astype(int)
C:\Users\Nastya\AppData\Local\Temp\ipykernel_8732\3574792755.py:9: SettingWithCopyWarning: 
A value is trying to b

In [420]:
df

,Game,Release_Date,Platforms,Score,Rating,Votes,Link_Steamspy,Owners,Imgs,Days_Since_Release,OwnersMin,OwnersMax
0,STAR WARS Empire at War Gold Pack,2006-02-16,[Windows],8.56,97,31401,https://steamspy.com/app/32470,1000000 .. 2000000,//steamcdn-a.akamaihd.net/steam/apps/32470/cap...,6275,1000000,2000000
1,Age of Empires II 2013,2013-04-09,"[Windows, Steam Deck]",8.51,96,97058,https://steamspy.com/app/221380,5000000 .. 10000000,//steamcdn-a.akamaihd.net/steam/apps/221380/ca...,3666,5000000,10000000
2,Stronghold Crusader HD,2002-07-31,[Windows],8.50,98,15306,https://steamspy.com/app/40970,1000000 .. 2000000,//steamcdn-a.akamaihd.net/steam/apps/40970/cap...,7571,1000000,2000000
3,Shadow Tactics Blades of the Shogun,2016-12-06,"[Windows, Mac, Linux, Steam Deck]",8.47,96,33604,https://steamspy.com/app/418240,1000000 .. 2000000,//steamcdn-a.akamaihd.net/steam/apps/418240/ca...,2329,1000000,2000000
4,Age of Empires II Definitive Edition,2019-11-14,[Windows],8.46,95,120875,https://steamspy.com/app/813780,2000000 .. 5000000,//steamcdn-a.akamaihd.net/steam/apps/813780/ca...,1256,2000000,5000000
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Breakout 13,2023-01-08,"[Windows, Mac, Linux]",7.60,87,4711,https://steamspy.com/app/2095300,100000 .. 200000,//steamcdn-a.akamaihd.net/steam/apps/624970/ca...,105,100000,200000
146,Natural Selection 2,2012-10-30,[Windows],7.60,85,12649,https://steamspy.com/app/4920,1000000 .. 2000000,//steamcdn-a.akamaihd.net/steam/apps/2095300/c...,3827,1000000,2000000
147,War for the Overworld,2015-04-02,"[Windows, Mac, Linux]",7.60,86,7153,https://steamspy.com/app/230190,500000 .. 1000000,//steamcdn-a.akamaihd.net/steam/apps/4920/caps...,2943,500000,1000000
148,Command Conquer Red Alert 3,2008-10-28,[Windows],7.59,85,11938,https://steamspy.com/app/17480,1000000 .. 2000000,//steamcdn-a.akamaihd.net/steam/apps/230190/ca...,5290,1000000,2000000


In [375]:
# df.to_excel(r'...\games1.xlsx')

In [389]:
# df_full = pd.read_excel(r'...\games1.xlsx', index_col=0)

In [393]:
# dfe = pd.read_csv(r'...games.csv')
dfee = dfe[dfe.Owners == '-']
dfee = dfee.drop(columns=['Unnamed: 0'])

dfee['Rating'] = dfee['Rating'].str.replace('%', '')
dfee['Votes'] = dfee['Votes'].str.replace('votes', '')
dfee['Rating'] = dfee['Rating'].astype(int)
dfee['Score'] = dfee['Score'].astype(float)
dfee['Release_Date'] = pd.to_datetime(dfee['Release_Date'], format='%d %b %Y')
dfee['Days_Since_Release'] = (dt.now() - dfee.Release_Date).dt.days

dfee.Votes = dfee.Votes.str.replace(',', '').astype(int)

In [394]:
dfee

,Game,Release_Date,Platforms,Score,Rating,Votes,Link_Steamspy,Owners,Days_Since_Release
10,Door Kickers 2 Task Force North,2020-11-03,['Windows'],8.36,97,6422,-,-,901
43,Heroes of the Three Kingdoms 2,2020-11-12,['Windows'],8.07,95,1276,-,-,892
63,Element TD 2 Tower Defense,2021-04-02,['Windows'],7.91,92,2282,-,-,751
64,From the Depths,2020-11-06,"['Windows', 'Mac', 'Linux']",7.90,90,9763,-,-,898
71,Banana Drama,2022-07-25,"['Windows', 'Mac', 'Linux']",7.86,97,207,-,-,272
74,ICBM,2020-11-17,['Windows'],7.85,92,1303,-,-,887
75,Heroes of the Three Kingdoms,2020-11-12,['Windows'],7.84,93,1050,-,-,892
97,112 Operator,2020-04-23,"['Windows', 'Mac']",7.74,89,4545,-,-,1095
115,Crustacean Nations,2022-08-19,['Windows'],7.69,94,262,-,-,247
116,Space Raiders in Space,2020-12-08,['Windows'],7.68,96,112,-,-,866


In [395]:
df_final = pd.concat([df_full, dfee])

In [396]:
df_final

,Game,Stylization,Platforms,Score,Rating,Votes,Link_Steamspy,Owners,Days_Since_Release,OwnersMin,OwnersMax,Release_Date
0,STAR WARS Empire at War Gold Pack,Semi-Realism,['Windows'],8.56,97,31401,https://steamspy.com/app/32470,1000000 .. 2000000,6275,1000000.0,2000000.0,2006-02-16
1,Age of Empires II 2013,Stylization,"['Windows', 'Steam Deck', 'PlayStation 2']",8.51,96,97058,https://steamspy.com/app/221380,5000000 .. 10000000,3666,5000000.0,10000000.0,2013-04-09
2,Stronghold Crusader HD,Stylization,['Windows'],8.50,98,15306,https://steamspy.com/app/40970,1000000 .. 2000000,7571,1000000.0,2000000.0,2002-07-31
3,Shadow Tactics Blades of the Shogun,Realism,"['Windows', 'Mac', 'Linux', 'Steam Deck', 'Pla...",8.47,96,33604,https://steamspy.com/app/418240,1000000 .. 2000000,2329,1000000.0,2000000.0,2016-12-06
4,Age of Empires II Definitive Edition,Realism,"['Windows', 'Xbox One']",8.46,95,120875,https://steamspy.com/app/813780,2000000 .. 5000000,1256,2000000.0,5000000.0,2019-11-14
...,...,...,...,...,...,...,...,...,...,...,...,...
115,Crustacean Nations,NaN,['Windows'],7.69,94,262,-,-,247,NaN,NaN,2022-08-19
116,Space Raiders in Space,NaN,['Windows'],7.68,96,112,-,-,866,NaN,NaN,2020-12-08
122,Burger Joint,NaN,"['Windows', 'Mac']",7.67,100,51,-,-,921,NaN,NaN,2020-10-14
129,Vedelem The Golden Horde,NaN,['Windows'],7.64,90,694,-,-,670,NaN,NaN,2021-06-22


In [3]:
# df_final.to_excel(r'...games_final.xlsx')
# df_finnall = pd.read_excel(r'...\games_final.xlsx', index_col=0)
import pandas as pd
df_finnall = pd.read_excel(r'F:\test_tasks\RetroStyle_Games\games_final.xlsx', index_col=0)

In [9]:
df_finnall['Votes_Per_Day'] = df_finnall['Votes']//df_finnall['Days_Since_Release']

In [12]:
df_finnall[df_finnall.Days_Since_Release < 180].sort_values('order', ascending=True)

,Game,Stylization,Platforms,Score,Rating,Votes,Link_Steamspy,Owners,Days_Since_Release,OwnersMin,OwnersMax,Release_Date,order,Votes_Per_Day
15,Against the Storm,Semi-Realism,['Windows'],8.31,95,9835,https://steamspy.com/app/1336490,200000 .. 500000,173,200000.0,500000.0,2022-11-01,15,56
142,cat and rat,Stylization,['Windows'],7.60,94,149,https://steamspy.com/app/2214950,0 .. 20000,162,0.0,20000.0,2022-11-12,130,0
145,Breakout 13,Realism,"['Windows', 'Mac', 'Linux']",7.60,87,4711,https://steamspy.com/app/2095300,100000 .. 200000,105,100000.0,200000.0,2023-01-08,133,44


In [13]:
df_finnall.sort_values(['Votes'],ascending=False)

,Game,Stylization,Platforms,Score,Rating,Votes,Link_Steamspy,Owners,Days_Since_Release,OwnersMin,OwnersMax,Release_Date,order,Votes_Per_Day
19,Hearts of Iron IV,Realism,"['Windows', 'Mac', 'Linux']",8.27,92,226948,https://steamspy.com/app/394360,5000000 .. 10000000,2512,5000000.0,10000000.0,2016-06-06,19,90
4,Age of Empires II Definitive Edition,Realism,"['Windows', 'Xbox One']",8.46,95,120875,https://steamspy.com/app/813780,2000000 .. 5000000,1256,2000000.0,5000000.0,2019-11-14,5,96
14,Total War WARHAMMER II,Semi-Realism,"['Windows', 'Mac', 'Linux']",8.31,93,119497,https://steamspy.com/app/594570,5000000 .. 10000000,2033,5000000.0,10000000.0,2017-09-28,14,58
1,Age of Empires II 2013,Stylization,"['Windows', 'Steam Deck', 'PlayStation 2']",8.51,96,97058,https://steamspy.com/app/221380,5000000 .. 10000000,3666,5000000.0,10000000.0,2013-04-09,2,26
25,Crusader Kings III,Realism,"['Windows', 'Mac', 'Linux', 'PlayStation 5', '...",8.24,92,88917,https://steamspy.com/app/1158310,2000000 .. 5000000,964,2000000.0,5000000.0,2020-09-01,25,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,Space Raiders in Space,Stylization,"['Windows', 'Xbox One', 'Xbox Series', 'Switch']",7.68,96,112,-,-,866,NaN,NaN,2020-12-08,147,0
113,It Stares Back,Stylization,['Windows'],7.69,98,86,https://steamspy.com/app/1094250,0 .. 20000,1398,0.0,20000.0,2019-06-25,106,0
136,IRON GUARD VR,Stylization,"['Windows', 'PlayStation 4']",7.61,96,84,https://steamspy.com/app/1213220,0 .. 20000,599,0.0,20000.0,2021-09-01,124,0
135,Byte Lynx,Stylization,['Windows'],7.62,98,59,https://steamspy.com/app/1996600,0 .. 20000,193,0.0,20000.0,2022-10-12,123,0


In [11]:
df_finnall.sort_values(['Votes_Per_Day'],ascending=False)

,Game,Stylization,Platforms,Score,Rating,Votes,Link_Steamspy,Owners,Days_Since_Release,OwnersMin,OwnersMax,Release_Date,order,Votes_Per_Day
4,Age of Empires II Definitive Edition,Realism,"['Windows', 'Xbox One']",8.46,95,120875,https://steamspy.com/app/813780,2000000 .. 5000000,1256,2000000.0,5000000.0,2019-11-14,5,96
25,Crusader Kings III,Realism,"['Windows', 'Mac', 'Linux', 'PlayStation 5', '...",8.24,92,88917,https://steamspy.com/app/1158310,2000000 .. 5000000,964,2000000.0,5000000.0,2020-09-01,25,92
92,Age of Empires IV Anniversary Edition,Realism,"['Windows', 'Xbox One']",7.77,86,49552,https://steamspy.com/app/1466860,2000000 .. 5000000,542,2000000.0,5000000.0,2021-10-28,86,91
19,Hearts of Iron IV,Realism,"['Windows', 'Mac', 'Linux']",8.27,92,226948,https://steamspy.com/app/394360,5000000 .. 10000000,2512,5000000.0,10000000.0,2016-06-06,19,90
14,Total War WARHAMMER II,Semi-Realism,"['Windows', 'Mac', 'Linux']",8.31,93,119497,https://steamspy.com/app/594570,5000000 .. 10000000,2033,5000000.0,10000000.0,2017-09-28,14,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Disney s Treasure Planet Battle of Procyon,Stylization,['Windows'],7.86,97,220,https://steamspy.com/app/331970,20000 .. 50000,7479,20000.0,50000.0,2002-10-31,69,0
114,Atom Zombie Smasher,Stylization,"['Windows', 'Mac', 'Linux']",7.69,89,1907,https://steamspy.com/app/55040,20000 .. 50000,4423,20000.0,50000.0,2011-03-14,107,0
46,Original War,Stylization,['Windows'],8.06,95,1773,https://steamspy.com/app/235320,200000 .. 500000,7982,200000.0,500000.0,2001-06-15,45,0
116,Space Raiders in Space,Stylization,"['Windows', 'Xbox One', 'Xbox Series', 'Switch']",7.68,96,112,-,-,866,NaN,NaN,2020-12-08,147,0
